# Trabajo final DMC

## 1. Recolección de datos con la API de Spotify

### Configuración de la API

In [2]:
# Librerías requeridas

## Spotipy
import spotipy
from spotipy.oauth2 import SpotifyOAuth

## Generales
from pprint import pprint

Inicialmente se debe configurar una aplicación en el *dashboard* de Spotify, según el [Web API tutorial](https://developer.spotify.com/documentation/web-api/quick-start/):

1. Ir a [developer.spotify.com](https://developer.spotify.com/)
2. Ir a la pestaña *DASHBOARD*
3. Ingresar con el nombre de usuario y contraseña de Spotify
4. Crear una nueva aplicación (*CREATE AN APP*), definiendo un nombre y agregando una descripción
5. *EDIT SETTINGS* -> Redirect URIs -> http://localhost:8080. Requerido para la autenticación desde Python
6. *USERS AND ACCESS* -> *ADD NEW USER* -> Definir nombre y correo electrónico de la cuenta de Spotify. Este será el usuario que podrá acceder a la API desde Python


### Conexión spotipy-app

In [3]:
scope = 'user-top-read'
cid = '923e41f1eb3f4f638b102398bb2bee6b'
secret = 'fc0ad2c973de49b99ce16d84e86f0f14'
redirect_uri = 'http://localhost:8080'

client_credentials_manager = SpotifyOAuth(client_id=cid, client_secret=secret, redirect_uri=redirect_uri, scope=scope)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=30)

In [4]:
# Mostrar el usuario actual
me = sp.me()
username = me['id']

pprint(me)
print('-'*10)
print('username:', username)

{'display_name': 'Jack Cruz',
 'external_urls': {'spotify': 'https://open.spotify.com/user/31prj7pij5zeghgoai4bws4bxshm'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/users/31prj7pij5zeghgoai4bws4bxshm',
 'id': '31prj7pij5zeghgoai4bws4bxshm',
 'images': [{'height': None,
             'url': 'https://platform-lookaside.fbsbx.com/platform/profilepic/?asid=536674957764087&height=300&width=300&ext=1681683115&hash=AeQjQCGcCwKHTjcEs0A',
             'width': None}],
 'type': 'user',
 'uri': 'spotify:user:31prj7pij5zeghgoai4bws4bxshm'}
----------
username: 31prj7pij5zeghgoai4bws4bxshm


### Listado de top-20 tracks                              

Como punto de partida se usarán los [user's top tracks](https://developer.spotify.com/documentation/web-api/reference/#endpoint-get-users-top-artists-and-tracks) que requiere privilegios tipo `user-top-read`.

Otros puntos de partida sugeridos:
- `sp.user_current_followed_artists`
- `sp.current_user_playlists`
- `sp.current_user_recently_played`
- `sp.current_user_top_artists`

entre otros.

In [5]:
# top20 = sp.current_user_recently_played()

toptracks = []
results = sp.current_user_top_tracks(time_range='short_term')
toptracks.extend(results['items'])

while results['next'] and len(toptracks) < 100:
    results = sp.next(results)
    toptracks.extend(results['items'])

# El resultado está almacenado en un diccionario en donde 'items'
# contiene la información de los tracks

for i, item in enumerate(toptracks):
    print(i, '. ', item['name'], '//', item['artists'][0]['name'], '//', item['preview_url'])

0 .  Pedindo Esmolas // Jésed // https://p.scdn.co/mp3-preview/794c7278f648fa0400b9f8daa3dbc765dc750150?cid=923e41f1eb3f4f638b102398bb2bee6b
1 .  Volví // Aventura // https://p.scdn.co/mp3-preview/4157475ff15a2340fcf5746fb7cdcb6dee7490ea?cid=923e41f1eb3f4f638b102398bb2bee6b
2 .  Promise (feat. Usher) // Romeo Santos // https://p.scdn.co/mp3-preview/60f8555a10c67bb6801f9ea842ba5d262e3b6645?cid=923e41f1eb3f4f638b102398bb2bee6b
3 .  De Ellos Aprendí // David Rees // https://p.scdn.co/mp3-preview/bdfebffb548a4897cf22d93bb30e565c7edd8216?cid=923e41f1eb3f4f638b102398bb2bee6b
4 .  Me Voy Enamorando // Chino & Nacho // None
5 .  Looking For Sade // Peabo Bryson // None
6 .  Into the Desert // Ely Guerra // https://p.scdn.co/mp3-preview/d543349f1e0798ab2e5c859fbce31d38920f605d?cid=923e41f1eb3f4f638b102398bb2bee6b
7 .  Gracias a Dios // Mach & Daddy // https://p.scdn.co/mp3-preview/6397e6686cdfd06078b21b1efacae5c63a3f512e?cid=923e41f1eb3f4f638b102398bb2bee6b
8 .  Higher Love // Kygo // https://p

#### Dataset del preferencias del usuario

Con estos tracks crearemos un dataset (en Pandas) que contendrá, por cada pista, su identificador y sus [características sonoras]():

In [6]:
import pandas as pd

# Extraer ids y nombres de las canciones, y extraer audio_features
my_track_ids = []
my_track_names = []
features = []
my_track_popularity = []

for track in toptracks:
    track_id = track['id']
    track_name = track['name']
    track_popularity = track['popularity']

    audio_features = sp.audio_features(track_id)
    
    my_track_ids.append(track_id)
    my_track_names.append(track_name)
    my_track_popularity.append(track_popularity)
    features.append(audio_features[0])
    
toptracks_df = pd.DataFrame(features, index=my_track_names)
toptracks_df['popularity'] = my_track_popularity
toptracks_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,popularity
Pedindo Esmolas,0.779,0.287,2,-14.267,0,0.0342,0.8120,0.000047,0.1700,0.697,117.982,audio_features,2ETyUK0F39pIYkkoQu6lRu,spotify:track:2ETyUK0F39pIYkkoQu6lRu,https://api.spotify.com/v1/tracks/2ETyUK0F39pI...,https://api.spotify.com/v1/audio-analysis/2ETy...,220721,3,5
Volví,0.725,0.860,1,-3.726,1,0.1820,0.4220,0.000284,0.0674,0.791,175.896,audio_features,2vmfvSoZBFAt9hhRoEByLi,spotify:track:2vmfvSoZBFAt9hhRoEByLi,https://api.spotify.com/v1/tracks/2vmfvSoZBFAt...,https://api.spotify.com/v1/audio-analysis/2vmf...,230125,4,77
Promise (feat. Usher),0.764,0.739,1,-6.422,1,0.0574,0.4030,0.000000,0.0821,0.770,132.995,audio_features,0jr6tT2vc4cIFPHb6wufG3,spotify:track:0jr6tT2vc4cIFPHb6wufG3,https://api.spotify.com/v1/tracks/0jr6tT2vc4cI...,https://api.spotify.com/v1/audio-analysis/0jr6...,252253,4,67
De Ellos Aprendí,0.720,0.572,7,-6.479,1,0.1160,0.1450,0.000000,0.4390,0.675,166.843,audio_features,1DnQiHRa2XnfwIf8qVl5C1,spotify:track:1DnQiHRa2XnfwIf8qVl5C1,https://api.spotify.com/v1/tracks/1DnQiHRa2Xnf...,https://api.spotify.com/v1/audio-analysis/1DnQ...,266076,4,56
Me Voy Enamorando,0.685,0.912,10,-3.411,1,0.0687,0.0538,0.000000,0.3920,0.511,99.957,audio_features,39Gb80bu5xF0rQtAxTxUS1,spotify:track:39Gb80bu5xF0rQtAxTxUS1,https://api.spotify.com/v1/tracks/39Gb80bu5xF0...,https://api.spotify.com/v1/audio-analysis/39Gb...,235720,4,55


In [7]:
# Reorganizar columnas
toptracks_df = toptracks_df[["id", "acousticness", "danceability", "duration_ms", "energy", "instrumentalness",  "key", "liveness", "loudness", "mode", "speechiness", "tempo", "valence"]]
toptracks_df.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
Pedindo Esmolas,2ETyUK0F39pIYkkoQu6lRu,0.8120,0.779,220721,0.287,0.000047,2,0.1700,-14.267,0,0.0342,117.982,0.697
Volví,2vmfvSoZBFAt9hhRoEByLi,0.4220,0.725,230125,0.860,0.000284,1,0.0674,-3.726,1,0.1820,175.896,0.791
Promise (feat. Usher),0jr6tT2vc4cIFPHb6wufG3,0.4030,0.764,252253,0.739,0.000000,1,0.0821,-6.422,1,0.0574,132.995,0.770
De Ellos Aprendí,1DnQiHRa2XnfwIf8qVl5C1,0.1450,0.720,266076,0.572,0.000000,7,0.4390,-6.479,1,0.1160,166.843,0.675
Me Voy Enamorando,39Gb80bu5xF0rQtAxTxUS1,0.0538,0.685,235720,0.912,0.000000,10,0.3920,-3.411,1,0.0687,99.957,0.511


#### Tracks candidatas

Partiendo de los top-20 tracks:
1. Extraeremos los artistas correspondientes (sin repeticiones)
2. Se usará la API para ampliar este listado a:
    - Artistas relacionados (`sp.artist_related_artists`)

Con este listado más amplio de artistas, obtenido en (1) y (2) se obtendrá:

3. El listado de albums
4. Y por cada album el listado de tracks

Y finalmente, para el listado resultante se obtendrán las *audio features*. El dataset resultante serán las pistas candidatas, que luego se llevarán al sistema de recomendación

In [8]:
# 1. Extraer los artistas correspondientes a las top-20 tracks (sin repeticiones)
ids_artists = []
print('Artistas de mi top:')
print('=====================')
for item in toptracks:
    artist_id = item['artists'][0]['id']
    artist_name = item['artists'][0]['name']
    print(f'{artist_id}: {artist_name}')
    ids_artists.append(artist_id)

# Depurar lista para evitar repeticiones
ids_artists = list(set(ids_artists))
print(f'Número de artistas (sin repeticiones): {len(ids_artists)}')

Artistas de mi top:
3TXp4wMgvRheVQPf1PEV2P: Jésed
1qto4hHid1P71emI6Fd8xi: Aventura
5lwmRuXgjX8xIwlnauTZIP: Romeo Santos
2CQHyfluB4mliyv193Qn2L: David Rees
5NS0854TqZQVoRmJKSWtFZ: Chino & Nacho
49iKbKGqgn8OESkW5WduX0: Peabo Bryson
1ne2c2YEgt4MmJCJGCsfsZ: Ely Guerra
5Q1vijNjTVhQBoVrZVXms8: Mach & Daddy
23fqKkggKUBHNkbKtXEls4: Kygo
6akDfyocmsREgR5eUXZt3I: Bely Basarte
2W8yFh0Ga6Yf3jiayVxwkE: Dove Cameron
6X8uzem5Dbl4l2so4CeexD: Nil Moliner
3nlHsNqwCSvT988ZfSW1Yh: Ronan Keating
4GNC7GD6oZMSxPGyXy4MNB: Lewis Capaldi
2xobqRIT4uGkrZVPJjRQeY: Mauricio Alen
60nua3AsVSfADZtg5Hdz3W: La Mosca Tse-Tse
4dpARuHxo51G3z768sgnrY: Adele
2MLHBMApNE5h8wIufiTPs7: Duncan Dhu
2urF8dgLVfDjunO0pcHUEe: Alvaro Soler
4TK1gDgb7QKoPFlzRrBRgR: Los Enanitos Verdes
7okwEbXzyT2VffBmyQBWLz: Maná
60uh2KYYSCqAgJNxcU4DA0: Hombres G
39BvzssARgDTZ1Kf0uqNfj: Té de Brujas
4TK1gDgb7QKoPFlzRrBRgR: Los Enanitos Verdes
13wFTN72PGSUxzEHJP5Ljs: Los Caligaris
7An4yvF7hDYDolN4m5zKBp: Soda Stereo
5VQCk9RiLwri99OgOT34kq: Vilma Palma e Va

In [9]:
# Por cada artista extraer sus top-3 tracks

id_tracks = []
nartists = len(ids_artists)
for i, id_artist in enumerate(ids_artists):
    print(f'Procesando artist {i+1} de {nartists}...')
    artist_tracks = sp.artist_top_tracks(id_artist)

    cnt = 0
    for track in artist_tracks['tracks']:
        if track['id'] in my_track_ids:
            continue
        
        cnt += 1
        id_tracks.append(track['id'])

        if cnt == 3:
            break
print(f'¡Listo! Número total de tracks pre-candidatos: {len(id_tracks)}')

Procesando artist 1 de 51...
Procesando artist 2 de 51...
Procesando artist 3 de 51...
Procesando artist 4 de 51...
Procesando artist 5 de 51...
Procesando artist 6 de 51...
Procesando artist 7 de 51...
Procesando artist 8 de 51...
Procesando artist 9 de 51...
Procesando artist 10 de 51...
Procesando artist 11 de 51...
Procesando artist 12 de 51...
Procesando artist 13 de 51...
Procesando artist 14 de 51...
Procesando artist 15 de 51...
Procesando artist 16 de 51...
Procesando artist 17 de 51...
Procesando artist 18 de 51...
Procesando artist 19 de 51...
Procesando artist 20 de 51...
Procesando artist 21 de 51...
Procesando artist 22 de 51...
Procesando artist 23 de 51...
Procesando artist 24 de 51...
Procesando artist 25 de 51...
Procesando artist 26 de 51...
Procesando artist 27 de 51...
Procesando artist 28 de 51...
Procesando artist 29 de 51...
Procesando artist 30 de 51...
Procesando artist 31 de 51...
Procesando artist 32 de 51...
Procesando artist 33 de 51...
Procesando artist 3

In [10]:
# En total se tendrán 543 (890) pistas candidatas. Para cada una se extraerán los
# audio_features y todo se almacenará en un dataframe

track_names = []
features = []
track_popularities = []

ntracks = len(id_tracks)
for i, track_id in enumerate(id_tracks):
    print(f'Procesando track {i+1} de {ntracks}...')
    track = sp.track(track_id)
    track_name = track['name']
    track_popularity = track['popularity']

    audio_features = sp.audio_features(track_id)
    
    #No incluir pistas sin "features"
    if audio_features[0] != None:
        track_names.append(track_name)
        features.append(audio_features[0])
        track_popularities.append(track_popularity)

print('¡Listo!')

candidatos_df = pd.DataFrame(features,index = track_names)
candidatos_df['popularity'] = track_popularities
candidatos_df.head()

Procesando track 1 de 153...
Procesando track 2 de 153...
Procesando track 3 de 153...
Procesando track 4 de 153...
Procesando track 5 de 153...
Procesando track 6 de 153...
Procesando track 7 de 153...
Procesando track 8 de 153...
Procesando track 9 de 153...
Procesando track 10 de 153...
Procesando track 11 de 153...
Procesando track 12 de 153...
Procesando track 13 de 153...
Procesando track 14 de 153...
Procesando track 15 de 153...
Procesando track 16 de 153...
Procesando track 17 de 153...
Procesando track 18 de 153...
Procesando track 19 de 153...
Procesando track 20 de 153...
Procesando track 21 de 153...
Procesando track 22 de 153...
Procesando track 23 de 153...
Procesando track 24 de 153...
Procesando track 25 de 153...
Procesando track 26 de 153...
Procesando track 27 de 153...
Procesando track 28 de 153...
Procesando track 29 de 153...
Procesando track 30 de 153...
Procesando track 31 de 153...
Procesando track 32 de 153...
Procesando track 33 de 153...
Procesando track 34

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,popularity
Estoy a la Puerta y Llamo,0.493,0.1120,10,-14.419,0,0.0345,0.935,0.000558,0.1630,0.243,76.480,audio_features,4bjsp9L4sWREYUkZU8nc6e,spotify:track:4bjsp9L4sWREYUkZU8nc6e,https://api.spotify.com/v1/tracks/4bjsp9L4sWRE...,https://api.spotify.com/v1/audio-analysis/4bjs...,198240,4,40
Nada Te Turbe,0.523,0.2420,0,-9.303,1,0.0269,0.859,0.000072,0.1050,0.319,103.928,audio_features,7GPBx7jwkMSnlUS6Qqudlp,spotify:track:7GPBx7jwkMSnlUS6Qqudlp,https://api.spotify.com/v1/tracks/7GPBx7jwkMSn...,https://api.spotify.com/v1/audio-analysis/7GPB...,216054,4,39
Entraré,0.426,0.0559,7,-18.357,1,0.0300,0.944,0.000000,0.0633,0.116,71.843,audio_features,1koXbetRP1PgmDRqU4DFPb,spotify:track:1koXbetRP1PgmDRqU4DFPb,https://api.spotify.com/v1/tracks/1koXbetRP1Pg...,https://api.spotify.com/v1/audio-analysis/1koX...,262373,4,38
Someone You Loved,0.501,0.4050,1,-5.679,1,0.0319,0.751,0.000000,0.1050,0.446,109.891,audio_features,7qEHsqek33rTcFNT9PFqLf,spotify:track:7qEHsqek33rTcFNT9PFqLf,https://api.spotify.com/v1/tracks/7qEHsqek33rT...,https://api.spotify.com/v1/audio-analysis/7qEH...,182161,4,86
Before You Go,0.459,0.5750,3,-4.858,1,0.0573,0.604,0.000000,0.0885,0.183,111.881,audio_features,2gMXnyrvIjhVBUZwvLZDMP,spotify:track:2gMXnyrvIjhVBUZwvLZDMP,https://api.spotify.com/v1/tracks/2gMXnyrvIjhV...,https://api.spotify.com/v1/audio-analysis/2gMX...,215107,4,82


In [11]:
candidatos_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,popularity
Estoy a la Puerta y Llamo,0.493,0.1120,10,-14.419,0,0.0345,0.935,0.000558,0.1630,0.243,76.480,audio_features,4bjsp9L4sWREYUkZU8nc6e,spotify:track:4bjsp9L4sWREYUkZU8nc6e,https://api.spotify.com/v1/tracks/4bjsp9L4sWRE...,https://api.spotify.com/v1/audio-analysis/4bjs...,198240,4,40
Nada Te Turbe,0.523,0.2420,0,-9.303,1,0.0269,0.859,0.000072,0.1050,0.319,103.928,audio_features,7GPBx7jwkMSnlUS6Qqudlp,spotify:track:7GPBx7jwkMSnlUS6Qqudlp,https://api.spotify.com/v1/tracks/7GPBx7jwkMSn...,https://api.spotify.com/v1/audio-analysis/7GPB...,216054,4,39
Entraré,0.426,0.0559,7,-18.357,1,0.0300,0.944,0.000000,0.0633,0.116,71.843,audio_features,1koXbetRP1PgmDRqU4DFPb,spotify:track:1koXbetRP1PgmDRqU4DFPb,https://api.spotify.com/v1/tracks/1koXbetRP1Pg...,https://api.spotify.com/v1/audio-analysis/1koX...,262373,4,38
Someone You Loved,0.501,0.4050,1,-5.679,1,0.0319,0.751,0.000000,0.1050,0.446,109.891,audio_features,7qEHsqek33rTcFNT9PFqLf,spotify:track:7qEHsqek33rTcFNT9PFqLf,https://api.spotify.com/v1/tracks/7qEHsqek33rT...,https://api.spotify.com/v1/audio-analysis/7qEH...,182161,4,86
Before You Go,0.459,0.5750,3,-4.858,1,0.0573,0.604,0.000000,0.0885,0.183,111.881,audio_features,2gMXnyrvIjhVBUZwvLZDMP,spotify:track:2gMXnyrvIjhVBUZwvLZDMP,https://api.spotify.com/v1/tracks/2gMXnyrvIjhV...,https://api.spotify.com/v1/audio-analysis/2gMX...,215107,4,82


In [12]:
candidatos_df = candidatos_df[["id", "acousticness", "danceability", "duration_ms", "energy", "instrumentalness",  "key", "liveness", "loudness", "mode", "speechiness", "tempo", "valence"]]

In [13]:
candidatos_df.shape

(153, 13)

In [14]:
candidatos_df.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
Estoy a la Puerta y Llamo,4bjsp9L4sWREYUkZU8nc6e,0.935,0.493,198240,0.1120,0.000558,10,0.1630,-14.419,0,0.0345,76.480,0.243
Nada Te Turbe,7GPBx7jwkMSnlUS6Qqudlp,0.859,0.523,216054,0.2420,0.000072,0,0.1050,-9.303,1,0.0269,103.928,0.319
Entraré,1koXbetRP1PgmDRqU4DFPb,0.944,0.426,262373,0.0559,0.000000,7,0.0633,-18.357,1,0.0300,71.843,0.116
Someone You Loved,7qEHsqek33rTcFNT9PFqLf,0.751,0.501,182161,0.4050,0.000000,1,0.1050,-5.679,1,0.0319,109.891,0.446
Before You Go,2gMXnyrvIjhVBUZwvLZDMP,0.604,0.459,215107,0.5750,0.000000,3,0.0885,-4.858,1,0.0573,111.881,0.183


In [15]:
toptracks_df.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
Pedindo Esmolas,2ETyUK0F39pIYkkoQu6lRu,0.8120,0.779,220721,0.287,0.000047,2,0.1700,-14.267,0,0.0342,117.982,0.697
Volví,2vmfvSoZBFAt9hhRoEByLi,0.4220,0.725,230125,0.860,0.000284,1,0.0674,-3.726,1,0.1820,175.896,0.791
Promise (feat. Usher),0jr6tT2vc4cIFPHb6wufG3,0.4030,0.764,252253,0.739,0.000000,1,0.0821,-6.422,1,0.0574,132.995,0.770
De Ellos Aprendí,1DnQiHRa2XnfwIf8qVl5C1,0.1450,0.720,266076,0.572,0.000000,7,0.4390,-6.479,1,0.1160,166.843,0.675
Me Voy Enamorando,39Gb80bu5xF0rQtAxTxUS1,0.0538,0.685,235720,0.912,0.000000,10,0.3920,-3.411,1,0.0687,99.957,0.511


In [18]:
# Guardar los resultados en un archivo csv
candidatos_df.to_csv('data/user_candidatos.csv')
toptracks_df.to_csv('data/user_toptracks.csv')

In [19]:
candidatos_df = pd.read_csv('data/user_candidatos.csv')
toptracks_df = pd.read_csv('data/user_toptracks.csv')

In [20]:
candidatos_df.head()

,Unnamed: 0,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,Estoy a la Puerta y Llamo,4bjsp9L4sWREYUkZU8nc6e,0.935,0.493,198240,0.1120,0.000558,10,0.1630,-14.419,0,0.0345,76.480,0.243
1,Nada Te Turbe,7GPBx7jwkMSnlUS6Qqudlp,0.859,0.523,216054,0.2420,0.000072,0,0.1050,-9.303,1,0.0269,103.928,0.319
2,Entraré,1koXbetRP1PgmDRqU4DFPb,0.944,0.426,262373,0.0559,0.000000,7,0.0633,-18.357,1,0.0300,71.843,0.116
3,Someone You Loved,7qEHsqek33rTcFNT9PFqLf,0.751,0.501,182161,0.4050,0.000000,1,0.1050,-5.679,1,0.0319,109.891,0.446
4,Before You Go,2gMXnyrvIjhVBUZwvLZDMP,0.604,0.459,215107,0.5750,0.000000,3,0.0885,-4.858,1,0.0573,111.881,0.183


In [21]:
candidatos_df.shape

(153, 14)

In [22]:
toptracks_df.head()

,Unnamed: 0,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,Pedindo Esmolas,2ETyUK0F39pIYkkoQu6lRu,0.8120,0.779,220721,0.287,0.000047,2,0.1700,-14.267,0,0.0342,117.982,0.697
1,Volví,2vmfvSoZBFAt9hhRoEByLi,0.4220,0.725,230125,0.860,0.000284,1,0.0674,-3.726,1,0.1820,175.896,0.791
2,Promise (feat. Usher),0jr6tT2vc4cIFPHb6wufG3,0.4030,0.764,252253,0.739,0.000000,1,0.0821,-6.422,1,0.0574,132.995,0.770
3,De Ellos Aprendí,1DnQiHRa2XnfwIf8qVl5C1,0.1450,0.720,266076,0.572,0.000000,7,0.4390,-6.479,1,0.1160,166.843,0.675
4,Me Voy Enamorando,39Gb80bu5xF0rQtAxTxUS1,0.0538,0.685,235720,0.912,0.000000,10,0.3920,-3.411,1,0.0687,99.957,0.511


In [23]:
toptracks_df.shape

(54, 14)